In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.datasets import cifar10
import numpy as np
import json
import os

In [2]:
#importing, scaling reshaping and vectorizing the data set
((trainX, trainY), (testX, testY)) = cifar10.load_data()

#scaling data
trainX = trainX.astype('float')/255.0
testX = testX.astype('float')/255.0


#vectorizing the labels
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.fit_transform(testY)

In [3]:
#initializing the label names
labelNames = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 
             'frog', 'horse', 'ship', 'truck']

In [4]:
#initializing the model
chanDim= -1
def model():
    
    model = Sequential()
    #first layer of Conv2D
    model.add(Conv2D(32, (3,3), padding = 'same', input_shape = (32, 32, 3)))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis = chanDim))
    model.add(Conv2D(32, (3,3), padding = 'same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis = chanDim))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Dropout(0.25))
    
    #second layer of Conv2D
    model.add(Conv2D(64, (3,3), padding = 'same', input_shape = (32, 32, 3)))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis = chanDim))
    model.add(Conv2D(64, (3,3), padding = 'same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis = chanDim))
    model.add(BatchNormalization(axis = chanDim))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Dropout(0.25))
    
    #FC layet
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(10))
    model.add(Activation('softmax'))
    
    return model

In [5]:
#initializing the optimizer and model
opt = SGD(lr = 0.01, decay = 0.01/40, momentum = 0.9, nesterov = True)
model = model()
model.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics = ['accuracy'])
model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
activation (Activation)      (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________

In [8]:
fname = os.path.sep.join(['weights', 
                          'weights -{epoch:03d} - {val_loss:.4f}.hdf5'])

checkpoint = ModelCheckpoint(fname, monitor = 'val_loss', 
                             mode = 'min', save_best_only = True, 
                             verbose = 1)
callbacks = [checkpoint]

H = model.fit(trainX, trainY, validation_data = (testX, testY), 
              batch_size = 64, epochs = 40, callbacks = callbacks, 
              verbose = 1)

Train on 50000 samples, validate on 10000 samples
Epoch 1/40
49984/50000 [============================>.] - ETA: 0s - loss: 0.9958 - acc: 0.6531
Epoch 00001: val_loss improved from inf to 0.85552, saving model to weights\weights -001 - 0.8555.hdf5
50000/50000 [==============================] - 11s 215us/sample - loss: 0.9958 - acc: 0.6531 - val_loss: 0.8555 - val_acc: 0.6993
Epoch 2/40
49920/50000 [============================>.] - ETA: 0s - loss: 0.8885 - acc: 0.6891
Epoch 00002: val_loss improved from 0.85552 to 0.79944, saving model to weights\weights -002 - 0.7994.hdf5
50000/50000 [==============================] - 11s 215us/sample - loss: 0.8882 - acc: 0.6892 - val_loss: 0.7994 - val_acc: 0.7167
Epoch 3/40
49984/50000 [============================>.] - ETA: 0s - loss: 0.8048 - acc: 0.7180
Epoch 00003: val_loss did not improve from 0.79944
50000/50000 [==============================] - 11s 228us/sample - loss: 0.8046 - acc: 0.7181 - val_loss: 0.8032 - val_acc: 0.7177
Epoch 4/40
499

Epoch 29/40
49920/50000 [============================>.] - ETA: 0s - loss: 0.3131 - acc: 0.8864
Epoch 00029: val_loss did not improve from 0.54848
50000/50000 [==============================] - 11s 230us/sample - loss: 0.3132 - acc: 0.8864 - val_loss: 0.5611 - val_acc: 0.8175
Epoch 30/40
49920/50000 [============================>.] - ETA: 0s - loss: 0.3028 - acc: 0.8917
Epoch 00030: val_loss did not improve from 0.54848
50000/50000 [==============================] - 11s 217us/sample - loss: 0.3029 - acc: 0.8917 - val_loss: 0.5541 - val_acc: 0.8169
Epoch 31/40
49920/50000 [============================>.] - ETA: 0s - loss: 0.2957 - acc: 0.8944
Epoch 00031: val_loss did not improve from 0.54848
50000/50000 [==============================] - 11s 219us/sample - loss: 0.2958 - acc: 0.8944 - val_loss: 0.5491 - val_acc: 0.8193
Epoch 32/40
49856/50000 [============================>.] - ETA: 0s - loss: 0.2896 - acc: 0.8982
Epoch 00032: val_loss did not improve from 0.54848
50000/50000 [=========